In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5),(0.5))
])

train = datasets.MNIST(
    root = './data',
    train = True,
    download = True,
    transform = transform
)
test = datasets.MNIST(
    root = './data',
    train = False,
    download = True,
    transform = transform
)

In [4]:
traini = DataLoader(train,batch_size=32,shuffle=True)
testi = DataLoader(test,batch_size=32,shuffle=False)

In [5]:
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1,8, kernel_size=3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(8,16,kernel_size=3, padding = 1)
        
        self.fc1 = nn.Linear(16*7*7, 64)
        self.fc2 = nn.Linear(64,10)

        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))

        x = x.view(x.size(0), -1)
        
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [6]:
model = CNN().to(device)
print(model)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)


In [7]:
criterion  = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
epochs  = 3

for epoch in range(epochs):
    model.train()

    for images, labels in traini :
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}] loss: {loss.item()/len(traini):.6f}')

Epoch [1] loss: 0.000016
Epoch [2] loss: 0.000011
Epoch [3] loss: 0.000022


In [9]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in testi:

        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct +=(predicted == labels).sum().item()

print(f'Accuracy :{100 * correct /total:.2f}%')

Accuracy :98.19%
